### UDACITY - FUNDAMENTOS DE DATA SCIENCE II  
#### PROJETO FINAL - Identificar fraude no Email da Enron

In [212]:
import sys, os, time
import pickle
sys.path.append("../tools/")

import tester
from feature_format import featureFormat, targetFeatureSplit
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

### Task 1: Select what features you'll use.
features_list is a list of strings, each of which is a feature name. The first feature must be "poi". You will need to use more features

In [213]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

# Atributo principal
main_feature = 'poi'

In [214]:
#Localiza características disponíveis
features_available = []
first_person = next(iter(data_dict))
for feature in data_dict[first_person]:
    if feature != 'poi': 
        features_available.append(feature)

In [215]:
# Lista de características
features_list = [main_feature] + features_available
n_features = len(features_list)

print 'Número total de características disponíveis:', n_features
print

print 'Características disponíveis:', features_list
print

Número total de características disponíveis: 21

Características disponíveis: ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'email_address', 'from_poi_to_this_person']



 Características disponíveis

 'poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options',
 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value',
 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees',
 'deferred_income', 'long_term_incentive', 'email_address', 'from_poi_to_this_person'

In [216]:
# Características selecionadas.
features_list = ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options',
                  'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value',
                  'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees',
                  'deferred_income', 'long_term_incentive','from_poi_to_this_person']

n_features = len(features_list)

print 'Número de características selecionados:', n_features
print

print 'Características selecionados:', features_list
print

Número de características selecionados: 20

Características selecionados: ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person']



In [217]:
def nan_check(data_dict):
    # Verifica existência de valores 'NaN' nas características selecionadas
    print '### Checando valores "NaN" ###'
    nan_dict = {}
    for key in features_list:
        nan_dict.update({key:0})

    for data_point in data_dict:
        for feature in data_dict[data_point]:
            if feature in features_list:
                if data_dict[data_point][feature] == 'NaN':
                    nan_dict[feature] += 1

    print 'Quantidade de valores "NaN" nas característica selecionada:'
    for feature in nan_dict:
        print feature+'= ', nan_dict[feature]
    print

In [218]:
def nan_to_zero(data_dict):
    # Troca valores NaN por 0
    print '### Trocando valores "NaN" por 0 ###'
    for data_point in data_dict:
        for feature in data_dict[data_point]:
            if data_dict[data_point][feature] == 'NaN':
                data_dict[data_point][feature] = 0
    print 'Concluído. OK!'
    print
    return data_dict

In [219]:
def financial_values_check(data_dict):
    # Verifica valores financeiros
    erro = 0
    print '### Checando se a soma dos valores de pagamento é igual ao total de valores de pagamento ###'
    for data_point in data_dict:
        total = (data_dict[data_point]['salary'] +
                 data_dict[data_point]['bonus'] +
                 data_dict[data_point]['long_term_incentive'] +
                 data_dict[data_point]['deferred_income'] +
                 data_dict[data_point]['deferral_payments'] +
                 data_dict[data_point]['loan_advances'] +
                 data_dict[data_point]['other'] +
                 data_dict[data_point]['expenses'] +
                 data_dict[data_point]['director_fees'])
        if total != data_dict[data_point]['total_payments']:
            erro += 1
            print 'Erro:', data_point
            print data_dict[data_point]
            print
    if erro == 0:
        print 'As somas dos valores de pagamento parecem certas agora.'
        print

In [220]:
# Verifica existência de valores 'NaN' nas características selecionadas
nan_check(data_dict)

# Troca valores NaN por 0
data_dict = nan_to_zero(data_dict)

# Verifica novamente a existência de valores 'NaN' nas características selecionadas
nan_check(data_dict)

### Checando valores "NaN" ###
Quantidade de valores "NaN" nas característica selecionada:
salary=  51
to_messages=  60
deferral_payments=  107
total_payments=  21
loan_advances=  142
bonus=  64
restricted_stock_deferred=  128
total_stock_value=  20
shared_receipt_with_poi=  60
long_term_incentive=  80
exercised_stock_options=  44
from_messages=  60
other=  53
from_poi_to_this_person=  60
from_this_person_to_poi=  60
poi=  0
deferred_income=  97
expenses=  51
restricted_stock=  36
director_fees=  129

### Trocando valores "NaN" por 0 ###
Concluído. OK!

### Checando valores "NaN" ###
Quantidade de valores "NaN" nas característica selecionada:
salary=  0
to_messages=  0
deferral_payments=  0
total_payments=  0
loan_advances=  0
bonus=  0
restricted_stock_deferred=  0
total_stock_value=  0
shared_receipt_with_poi=  0
long_term_incentive=  0
exercised_stock_options=  0
from_messages=  0
other=  0
from_poi_to_this_person=  0
from_this_person_to_poi=  0
poi=  0
deferred_income=  0
expenses=

In [221]:
# Conta número data points
num_data_points = 0
for data_points in data_dict.iteritems():
    num_data_points += 1
    name = data_points[0]

print
print 'Número de data points:', num_data_points
print

# Conta número de POIs e Não-POIs
num_poi = 0
num_npoi = 0
for data_point in data_dict:
    if data_dict[data_point]['poi']:
        num_poi += 1
    else:
        num_npoi += 1

print 'Total de POIs    :', num_poi
print 'Total de Não-POIs:', num_npoi
print


Número de data points: 146

Total de POIs    : 18
Total de Não-POIs: 128



In [222]:
# Verifica a soma dos valores de pagamento
financial_values_check(data_dict)

print 'Foi detectada uma diferença nos valore financeiros.'
print 'Ao verificar o arquivo "enron1702insiderpay.pdf" ficou claro que houve um deslocamento das informações.'
print 'Possivelmente algum erro humano na entrada das informações no conjunto de dados.'
print 'O erro foi corrigido programaticamente.'
print

data_dict['BELFER ROBERT']['deferred_income'] = data_dict['BELFER ROBERT']['deferral_payments']
data_dict['BELFER ROBERT']['deferral_payments'] = 0
data_dict['BELFER ROBERT']['expenses'] = data_dict['BELFER ROBERT']['director_fees']
data_dict['BELFER ROBERT']['director_fees'] = data_dict['BELFER ROBERT']['total_payments']
data_dict['BELFER ROBERT']['total_payments'] = data_dict['BELFER ROBERT']['exercised_stock_options']
data_dict['BELFER ROBERT']['exercised_stock_options'] = 0

print 'Dados referentes a BELFER ROBERT corrigidos!'
data_dict['BELFER ROBERT']
print

data_dict['BHATNAGAR SANJAY']['total_stock_value'] = data_dict['BHATNAGAR SANJAY']['restricted_stock_deferred']
data_dict['BHATNAGAR SANJAY']['restricted_stock_deferred'] = data_dict['BHATNAGAR SANJAY']['restricted_stock']
data_dict['BHATNAGAR SANJAY']['restricted_stock'] = data_dict['BHATNAGAR SANJAY']['exercised_stock_options']
data_dict['BHATNAGAR SANJAY']['exercised_stock_options'] = data_dict['BHATNAGAR SANJAY']['total_payments']
data_dict['BHATNAGAR SANJAY']['total_payments'] = data_dict['BHATNAGAR SANJAY']['director_fees']
data_dict['BHATNAGAR SANJAY']['director_fees'] = data_dict['BHATNAGAR SANJAY']['expenses']
data_dict['BHATNAGAR SANJAY']['expenses'] = data_dict['BHATNAGAR SANJAY']['other']
data_dict['BHATNAGAR SANJAY']['other'] = 0

print 'Dados referentes a BHATNAGAR SANJAY corrigidos!'
data_dict['BHATNAGAR SANJAY']
print

# Verifica valores de pagamento
financial_values_check(data_dict)

### Checando se a soma dos valores de pagamento é igual ao total de valores de pagamento ###
Erro: BELFER ROBERT
{'salary': 0, 'to_messages': 0, 'deferral_payments': -102500, 'total_payments': 102500, 'exercised_stock_options': 3285, 'bonus': 0, 'restricted_stock': 0, 'shared_receipt_with_poi': 0, 'restricted_stock_deferred': 44093, 'total_stock_value': -44093, 'expenses': 0, 'loan_advances': 0, 'from_messages': 0, 'other': 0, 'from_this_person_to_poi': 0, 'poi': False, 'director_fees': 3285, 'deferred_income': 0, 'long_term_incentive': 0, 'email_address': 0, 'from_poi_to_this_person': 0}

Erro: BHATNAGAR SANJAY
{'salary': 0, 'to_messages': 523, 'deferral_payments': 0, 'total_payments': 15456290, 'exercised_stock_options': 2604490, 'bonus': 0, 'restricted_stock': -2604490, 'shared_receipt_with_poi': 463, 'restricted_stock_deferred': 15456290, 'total_stock_value': 0, 'expenses': 0, 'loan_advances': 0, 'from_messages': 29, 'other': 137864, 'from_this_person_to_poi': 1, 'poi': False, 'dir

### Task 2: Remove outliers

In [223]:
def scatter_salary_bonus(data_dict):
    # Plota gráfico Salary x Bonus
    for data_point in data_dict:
        salary = data_dict[data_point]['salary']
        bonus = data_dict[data_point]['bonus']
        #if salary != 'NaN' and bonus != 'NaN':
        plt.scatter( salary, bonus )

    plt.xlabel("salary")
    plt.ylabel("bonus")
    plt.show()

In [224]:
def greater_salary(data_dict):
    # Localiza data ponit com maior salário
    print '### Localizando maior salário ###'
    person_greater_salary = []
    max_salary = 0
    for data_point in data_dict:
        salary = data_dict[data_point]['salary']
        #if salary != 'NaN' and 
        if salary > max_salary:
            max_salary = salary
            person_greater_salary = data_point, '=', data_dict[data_point]
    print 'Data point onde ocorre o maior salário:'
    print person_greater_salary
    print

In [225]:
def remove_outliers(outlier):
    # Remove outliers
    data_dict.pop(outlier, 0)
    print 'Outlier removido:', outlier
    print

#### Mark Frevert  
Atuou como diretor executivo da Enron Europe de 1986 a 2000 e foi nomeado presidente da Enron em 2001.  
Ele era um integrante importante da empresa, mas não foi considerado uma pessoa de interesse.  
Acredito que ele era um funcionário acima média da Enron durante esse período por causa de sua compensação substancial e o removerá do conjunto de dados.

#### John Baxter  
Era um ex-vice-presidente da Enron e morreu de um aparente disparo auto-infligido antes de poder testemunhar contra outros executivos da Enron. Vou removê-lo do conjunto de dados, pois ele não é uma pessoa de interesse.

#### John Lavorato  
Era um executivo de alto escalão na divisão de comércio de energia da Enron e recebeu grandes bônus para não deixar a Enron.  
Como ele não era uma pessoa de interesse, e o grande bônus acabou distorcendo sua remuneração total, acho que seria apropriado removê-lo do conjunto de dados.

#### Lawrence Whalley  
Serviu como presidente da Enron e demitiu Andrew Fastow quando ficou evidente a gravidade da situação da Enron. Ele foi investigado completamente, mas não identificado como uma pessoa de interesse e, portanto, será removido do conjunto de dados.

In [226]:
# Verifica atributos Salary e Bonus
print 'No gráfico plotado foi percebido alguns pontos fora do padrão.'
print
#scatter_salary_bonus(data_dict)

greater_salary(data_dict)
print 'Ao checar o maior valor foi percebido um data point não útil para a análise com nome "TOTAL"'
print 'Optou-se por remover o outlier.'

remove_outliers('TOTAL')

# Verifica atributos Salary e Bonus

print 'Novo gráfico após a remoção do outlier "TOTAL".'
print
#scatter_salary_bonus(data_dict)

greater_salary(data_dict)
print 'O maior salário agora é relativo a uma pessoa.'
print

print 'Ao observar o arquivo "enron61702insiderpay.pdf" foi percebido um data point não referente a uma pessoa.'
print 'Por não parecer útil para esta análise optou-se por removê-lo: "THE TRAVEL AGENCY IN THE PARK"'
print
remove_outliers('THE TRAVEL AGENCY IN THE PARK')

remove_outliers('FREVERT MARK A')
remove_outliers('LAVORATO JOHN J')
remove_outliers('WHALLEY LAWRENCE G')
remove_outliers('BAXTER JOHN C')

No gráfico plotado foi percebido alguns pontos fora do padrão.

### Localizando maior salário ###
Data point onde ocorre o maior salário:
('TOTAL', '=', {'salary': 26704229, 'to_messages': 0, 'deferral_payments': 32083396, 'total_payments': 309886585, 'exercised_stock_options': 311764000, 'bonus': 97343619, 'restricted_stock': 130322299, 'shared_receipt_with_poi': 0, 'restricted_stock_deferred': -7576788, 'total_stock_value': 434509511, 'expenses': 5235198, 'loan_advances': 83925000, 'from_messages': 0, 'other': 42667589, 'from_this_person_to_poi': 0, 'poi': False, 'director_fees': 1398517, 'deferred_income': -27992891, 'long_term_incentive': 48521928, 'email_address': 0, 'from_poi_to_this_person': 0})

Ao checar o maior valor foi percebido um data point não útil para a análise com nome "TOTAL"
Optou-se por remover o outlier.
Outlier removido: TOTAL

Novo gráfico após a remoção do outlier "TOTAL".

### Localizando maior salário ###
Data point onde ocorre o maior salário:
('SKILLING JEF

### Task 3: Create new feature(s)  
Store to my_dataset for easy export below.

In [227]:
my_dataset = data_dict

In [228]:
# Normalização

df = pd.DataFrame.from_dict(my_dataset, orient='index')
df = df[features_list]

df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 20 columns):
poi                          140 non-null bool
salary                       140 non-null int64
to_messages                  140 non-null int64
deferral_payments            140 non-null int64
total_payments               140 non-null int64
exercised_stock_options      140 non-null int64
bonus                        140 non-null int64
restricted_stock             140 non-null int64
shared_receipt_with_poi      140 non-null int64
restricted_stock_deferred    140 non-null int64
total_stock_value            140 non-null int64
expenses                     140 non-null int64
loan_advances                140 non-null int64
from_messages                140 non-null int64
other                        140 non-null int64
from_this_person_to_poi      140 non-null int64
director_fees                140 non-null int64
deferred_income              140 non-null int64
long_term_incenti

In [229]:
scaled_df = df.copy()
scaled_df.iloc[:,1:] = scale(scaled_df.iloc[:,1:])

In [230]:
my_dataset = scaled_df.to_dict(orient='index')

In [231]:
# Adiciona novas caracteristicas 
use_new_feature = True

if use_new_feature:
    #for person in my_dataset.values():
    for person in my_dataset:
        my_dataset[person]['to_poi_message_proportion'] = my_dataset[person]['from_this_person_to_poi']/my_dataset[person]['to_messages']
        my_dataset[person]['from_poi_message_proportion'] = my_dataset[person]['from_poi_to_this_person']/my_dataset[person]['from_messages']
        my_dataset[person]['shared_poi_message_proportion'] = my_dataset[person]['shared_receipt_with_poi']/my_dataset[person]['to_messages']
        my_dataset[person]['bonus_to_salary'] = my_dataset[person]['bonus'] / my_dataset[person]['salary']
        my_dataset[person]['bonus_to_total'] = my_dataset[person]['bonus'] / my_dataset[person]['total_payments']
        
    features_list = features_list + ['to_poi_message_proportion', 'from_poi_message_proportion', 'shared_poi_message_proportion', 'bonus_to_salary', 'bonus_to_total']

    print 'Lista com novos atributos:', features_list
    print
    print 'Total de atributos:', len(features_list)
    print

Lista com novos atributos: ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person', 'to_poi_message_proportion', 'from_poi_message_proportion', 'shared_poi_message_proportion', 'bonus_to_salary', 'bonus_to_total']

Total de atributos: 25



In [232]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

### Task 4: Try a varity of classifiers
Please name your classifier clf for easy export below.
Note that if you want to do PCA or other multi-stage operations,
you'll need to use Pipelines. For more info:
http://scikit-learn.org/stable/modules/pipeline.html

In [233]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [236]:
# Decision Tree Classifier
skb = SelectKBest()
dtree = DecisionTreeClassifier()

pipe = Pipeline(steps=[('skb', skb), ('dtree', dtree)])

k = n_features = np.arange(1, len(features_list))

clf = GridSearchCV(pipe, dict(skb__k=k), scoring='f1', cv =10)
print
print 'Decision Tree Classifier'
print
clf.fit(features, labels)

print
print clf.best_params_
print
print clf.best_score_
print


Decision Tree Classifier


{'skb__k': 5}

0.4926190476190476



In [238]:
# AdaBoost Classifier
skb = SelectKBest()
ada = AdaBoostClassifier()

pipe = Pipeline(steps=[('skb', skb), ('ada', ada)])

k = n_features = np.arange(1, len(features_list))

clf = GridSearchCV(pipe, dict(skb__k=k), scoring='f1', cv =10)
print
print 'AdaBoost Classifier'
print
clf.fit(features, labels)

print
print clf.best_score_
print
print clf.best_params_
print


AdaBoost Classifier


0.4345238095238095

{'skb__k': 13}



### Task 5: Tune your classifier to achieve better than .3 precision and recall 
using our testing script. Check the tester.py script in the final project
folder for details on the evaluation method, especially the test_classifier
function. Because of the small size of the dataset, the script uses
stratified shuffle split cross validation. For more info: 
http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [242]:
# Decision Tree Classifier
skb = SelectKBest()
dtree = DecisionTreeClassifier()

pipe = Pipeline(steps=[('skb', skb), ('dtree', dtree)])

k = n_features = [5]
cri = ['gini','entropy']
m_d = [None, 5, 10, 15, 20]
m_f = [None, 'sqrt', 'log2', 'auto']
m_s_s = [2, 4, 6, 8, 10, 20]

dtree_clf = GridSearchCV(pipe, dict(skb__k=k, dtree__criterion=cri), scoring='f1', cv =10)

print '### Decision Tree Classifier ###'
print
dtree_clf.fit(features, labels)
print
print dtree_clf.best_score_
print
print dtree_clf.best_params_
print

### Decision Tree Classifier ###


0.4235714285714286

{'dtree__criterion': 'gini', 'skb__k': 5}



print 'Iniciando Tester...'
print
tester.test_classifier(dtree_clf, my_dataset, features_list, folds = 200)
print

In [245]:
# AdaBoost Classifier
skb = SelectKBest()
ada = AdaBoostClassifier()

pipe = Pipeline(steps=[('skb', skb), ('ada', ada)])

k = n_features = [13]
b_est = [DecisionTreeClassifier(), RandomForestClassifier()]#, GaussianNB()]
n_est = [30, 50, 70, 120]
l_rat = [0.5, 1, 1.5, 2, 4]

ada_clf = GridSearchCV(pipe, dict(skb__k=k, ada__base_estimator=b_est, ada__n_estimators=n_est, ada__learning_rate=l_rat), scoring='f1', cv =10)

print '### AdaBoost Classifier ###'
print
ada_clf.fit(features, labels)
print
print ada_clf.best_score_
print
print ada_clf.best_params_
print

### AdaBoost Classifier ###


0.43999999999999995

{'ada__n_estimators': 70, 'ada__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'ada__learning_rate': 0.5, 'skb__k': 13}



print 'Iniciando Tester...'
print
tester.test_classifier(ada_clf, my_dataset, features_list, folds = 200)
print

In [ ]:
# Parâmetros otimizados para DecisionTree
dtree_clf = Pipeline([('select_features', SelectKBest(k=10)),
                      ('classify', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                                                          max_features=None, max_leaf_nodes=None,
                                                          min_impurity_decrease=0.0, min_impurity_split=None,
                                                          min_samples_leaf=1, min_samples_split=2,
                                                          min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                                                          splitter='best')
                      )])

# Parâmetros otimizados para AdaBoost
ada_clf = Pipeline([('select_features', SelectKBest(k=13)),
                    ('classify', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini',
                                                    max_depth=None,
                                                    max_features=None, max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0, min_impurity_split=None,
                                                    min_samples_leaf=1, min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                                                    splitter='best'), learning_rate=0.5, n_estimators=70)
                    )])

# Classificador escolhido
clf = dtree_clf

### Task 6: Dump your classifier, dataset, and features_list so anyone can
check your results. You do not need to change anything below, but make sure
that the version of poi_id.py that you submit can be run on its own and
generates the necessary .pkl files for validating your results.

In [ ]:
tester.dump_classifier_and_data(clf, my_dataset, features_list)

print 'Iniciando Tester...'
print
tester.main()
print